In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from experiment import experiment

In [3]:
import os

In [4]:
import pandas as pd

In [5]:
results = os.listdir('results')

In [6]:
results = [result for result in results if result.endswith('.pkl') and "5.31.2023" in result and "rfqr_20runs" in result and "std" in result and "B=100" in result and "randomConformalTrue" in result and "logFalse" in result]

In [7]:
results

['bio_rfqr_20runs_0.25fraction_logFalse_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_05.31.2023.pkl',
 'homes_rfqr_20runs_0.5fraction_logFalse_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_05.31.2023.pkl',
 'concrete_rfqr_20runs_1fraction_logFalse_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_05.31.2023.pkl',
 'bike_rfqr_20runs_1fraction_logFalse_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_05.31.2023.pkl',
 'community_rfqr_20runs_1fraction_logFalse_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_05.31.2023.pkl',
 'star_rfqr_20runs_1fraction_logFalse_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_05.31.2023.pkl']

In [8]:
output = pd.DataFrame(columns=['UACQR-P','UACQR-S', 'CQR','CQR-r', 'UACQR-P SE', 'UACQR-S SE', 'CQR SE','CQR-r SE'])
for i,result in enumerate(results):
    title = results[i].split('_')[0] if results[i].split('_')[0] not in ["facebook","meps"] else '_'.join(results[i].split('_')[0:2])
    result = experiment(file_name='results/'+results[i])
    result.plot(calc_only=True)
    output.loc[title,['UACQR-P','UACQR-S', 'CQR','CQR-r']] = result.mean_results.min()
    output.loc[title,['UACQR-P SE','UACQR-S SE', 'CQR SE','CQR-r SE']] = result.sem_results.stack().loc[pd.MultiIndex.from_frame(result.mean_results.idxmin().rename('min_samples_leaf').reset_index().iloc[:,::-1])].to_numpy()
    
    

In [9]:
output.sort_index()

,UACQR-P,UACQR-S,CQR,CQR-r,UACQR-P SE,UACQR-S SE,CQR SE,CQR-r SE
bike,1.143357,1.451945,1.480517,1.435097,0.008622,0.007771,0.007501,0.008011
bio,1.534535,1.649856,1.65199,1.648958,0.008387,0.005643,0.005583,0.005667
community,1.493758,1.750159,1.760516,1.756741,0.021019,0.017161,0.015618,0.016135
concrete,0.667608,0.728981,0.745444,0.720602,0.008121,0.007627,0.006917,0.008211
homes,0.625816,0.692549,0.753587,0.707159,0.004544,0.00495,0.004686,0.00508
star,0.174783,0.174528,0.174299,0.174519,0.000826,0.000776,0.000726,0.000792


In [10]:
from functools import partial

In [11]:
def format_fig_and_se(row, method='CQR'):
    fig = row[method]
    se = row[method+' SE']
    if fig >1000:
        return f"{fig:.0f} ({se:.0f})"
    elif fig >1:
        return f"{fig:.3f} ({se:.3f})"
    else:
        return f"{fig:.3f} ({se:.3f})"

In [12]:
drop_columns = [c for c in output.columns if 'SE' in c]

In [13]:
output['CQR'] = output.apply(partial(format_fig_and_se, method='CQR'), axis=1)
output['UACQR-S'] = output.apply(partial(format_fig_and_se, method='UACQR-S'), axis=1)
output['UACQR-P'] = output.apply(partial(format_fig_and_se, method='UACQR-P'), axis=1)
output['CQR-r'] = output.apply(partial(format_fig_and_se, method='CQR-r'), axis=1)

In [14]:
output.drop(columns=drop_columns, inplace=True)

In [15]:
# Export to LaTeX
latex_table = output.sort_index().style.to_latex()


print('\_'.join(latex_table.split('_')))

\begin{tabular}{lllll}
 & UACQR-P & UACQR-S & CQR & CQR-r \\
bike & 1.143 (0.009) & 1.452 (0.008) & 1.481 (0.008) & 1.435 (0.008) \\
bio & 1.535 (0.008) & 1.650 (0.006) & 1.652 (0.006) & 1.649 (0.006) \\
community & 1.494 (0.021) & 1.750 (0.017) & 1.761 (0.016) & 1.757 (0.016) \\
concrete & 0.668 (0.008) & 0.729 (0.008) & 0.745 (0.007) & 0.721 (0.008) \\
homes & 0.626 (0.005) & 0.693 (0.005) & 0.754 (0.005) & 0.707 (0.005) \\
star & 0.175 (0.001) & 0.175 (0.001) & 0.174 (0.001) & 0.175 (0.001) \\
\end{tabular}

